In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from tensorflow.contrib.layers.python.layers import instance_norm
import random
import os
import time

In [ ]:
'''Load Dataset'''
os.environ['CUDA_VISIBLE_DEVICES']='0'
question=np.load(r'dataset\question.npy')
answer=np.load(r'dataset\answer.npy')
answer=answer[np.sum(question>0,(1,2))<=20]
question=question[np.sum(question>0,(1,2))<=20]

'''Data Augmentation'''
def shuffle(que,ans):
    li=np.arange(0,9).reshape([3,3])
    np.random.shuffle(li)
    for _ in li:
        np.random.shuffle(_)
    li=li.reshape([-1])

    il=np.arange(0,9).reshape([3,3])
    np.random.shuffle(il)
    for _ in il:
        np.random.shuffle(_)
    il=il.reshape([-1])
    que=que[:,li,:][:,:,il]+10
    ans=ans[:,li,:][:,:,il]+10
    que[que==10]=0
    numtra=np.arange(11,20)
    np.random.shuffle(numtra)
    for n,_ in enumerate(numtra):
        que[que==_]=n+1
        ans[ans==_]=n+1
    if np.random.randint(0,2):
        que=que.transpose(0,2,1)
        ans=ans.transpose(0,2,1)
    return que,ans

num=0
def next_batch(batch_size=128,num_data=180000):
    global num
    begin=num
    end=num+batch_size
    num=end
    if(end>=num_data-1):
        end=num_data-1
        num=0
    q,a=shuffle(question[begin:end],answer[begin:end])
    q=np.reshape(q,[-1,9,9,1])==np.arange(0,10)
    a=np.reshape(a,[-1,9,9,1])==np.arange(1,10)
    return q.astype(np.float32),a.astype(np.float32)

'''Normalization Function'''
def bn(x):
    x=instance_norm(x, epsilon=1e-05)
    return x

'''Linear Function'''
def line(x):
    return x

'''conv'''
def conv(x,filters,kernel_size=3,stride=1,activation=tf.nn.leaky_relu, padding='valid',norm=True,bias=False):
    with tf.name_scope('conv'):
        x=tf.layers.conv2d(x,filters,kernel_size=kernel_size, strides=stride, padding=padding,use_bias=bias)
        if norm:
            x=bn(x)
            x=activation(x)
    return x

'''dconv'''
def dconv(x,filters,kernel_size=3,stride=1,activation=tf.nn.leaky_relu,padding='valid',norm=True,bias=False):
    with tf.name_scope('dconv'):
        x=tf.layers.conv2d_transpose(x,filters,kernel_size=kernel_size, strides=stride, padding=padding,use_bias=bias)
        if norm:
            x=bn(x)
            x=activation(x)
    return x

'''SDconv'''
def sdconv(x,filters,activation=tf.nn.leaky_relu):
    x1=conv(x,filters*2,kernel_size=[1,3],stride=(1,3),activation=activation)
    x2=conv(x,filters*2,kernel_size=[3,1],stride=(3,1),activation=activation)
    x3=x
    x13=dconv(x1,filters*1,kernel_size=[1,1],activation=activation,stride=1)
    x23=dconv(x2,filters*1,kernel_size=[1,1],activation=activation,stride=1)
    
    x11=conv(x1,filters*2,kernel_size=[1,3],stride=(1,3),activation=activation)
    x12=conv(x1,filters*1,kernel_size=[3,1],stride=(3,1),activation=activation)
    x21=conv(x2,filters*1,kernel_size=[1,3],stride=(1,3),activation=activation)
    x22=conv(x2,filters*2,kernel_size=[3,1],stride=(3,1),activation=activation)
    x12=tf.concat([x12,x21],-1)
    x113=dconv(x11,filters*1,kernel_size=[1,1],activation=activation,stride=1)
    x123=dconv(x12,filters*1,kernel_size=[1,1],activation=activation,stride=1)
    x223=dconv(x22,filters*1,kernel_size=[1,1],activation=activation,stride=1)
    
    x112=conv(x11,filters*2,kernel_size=[3,1],stride=(3,1),activation=activation)
    x121=conv(x12,filters*2,kernel_size=[1,3],stride=(1,3),activation=activation)
    x212=conv(x12,filters*2,kernel_size=[3,1],stride=(3,1),activation=activation)
    x221=conv(x22,filters*2,kernel_size=[1,3],stride=(1,3),activation=activation)
    x121=tf.concat([x112,x121],-1)
    x212=tf.concat([x212,x221],-1)
    x1213=dconv(x121,filters*2,kernel_size=[1,1],activation=activation,stride=1)
    x2123=dconv(x212,filters*2,kernel_size=[1,1],activation=activation,stride=1)
    
    x1212=conv(x121,filters*4,kernel_size=[3,1],stride=(3,1),activation=activation)
    x2121=conv(x212,filters*4,kernel_size=[1,3],stride=(1,3),activation=activation)
    x1212=tf.concat([x1212,x2121],-1)
    
    x121=dconv(x1212,filters*2,kernel_size=[3,1],stride=(3,1),activation=activation)
    x212=dconv(x1212,filters*2,kernel_size=[1,3],stride=(1,3),activation=activation)
    x121=tf.concat([x121,x1213],-1)
    x212=tf.concat([x212,x2123],-1)
    
    x11=dconv(x121,filters*1,kernel_size=[3,1],stride=(3,1),activation=activation)
    x12=dconv(x121,filters*1,kernel_size=[1,3],stride=(1,3),activation=activation)
    x21=dconv(x212,filters*1,kernel_size=[3,1],stride=(3,1),activation=activation)
    x22=dconv(x212,filters*1,kernel_size=[1,3],stride=(1,3),activation=activation)
    x11=tf.concat([x11,x113],-1)
    x12=tf.concat([x12,x21,x123],-1)
    x22=tf.concat([x22,x223],-1)
    
    x1=dconv(x11,filters*1,kernel_size=[1,3],stride=(1,3),activation=activation)
    x1_=dconv(x12,filters*1,kernel_size=[3,1],stride=(3,1),activation=activation)
    x2=dconv(x12,filters*1,kernel_size=[1,3],stride=(1,3),activation=activation)
    x2_=dconv(x22,filters*1,kernel_size=[3,1],stride=(3,1),activation=activation)
    x1=tf.concat([x1,x1_,x13],-1)
    x2=tf.concat([x2,x2_,x23],-1)
    
    x=dconv(x1,int(filters*1),kernel_size=[1,3],stride=(1,3),activation=line,norm=False)
    x_=dconv(x2,int(filters*1),kernel_size=[3,1],stride=(3,1),activation=line,norm=False)

    x=tf.concat([x3,x,x_],-1)
    x=activation(bn(x))
    x=dconv(x,int(filters*1),kernel_size=[1,1],activation=line,stride=1,norm=False)
    return x


In [ ]:
def sdconv0(net):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('step0')])>0
    with tf.variable_scope('step0',reuse=reuse):
        net=sdconv(net,256)
    return net

def sdconv1(net):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('step1')])>0
    with tf.variable_scope('step1',reuse=reuse):
        net=sdconv(net,256)
    return net

def sdconv2(net):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('step2')])>0
    with tf.variable_scope('step2',reuse=reuse):
        net=sdconv(net,256)
    return net

def sdconv3(net):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('step3')])>0
    with tf.variable_scope('step3',reuse=reuse):
        net=sdconv(net,256)
    return net

def sdconv4(net):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('step4')])>0
    with tf.variable_scope('step4',reuse=reuse):
        net=sdconv(net,256)
    return net

def sdconv5(net):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('step5')])>0
    with tf.variable_scope('step5',reuse=reuse):
        net=sdconv(net,256)
    return net


def sdconv6(net):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('step6')])>0
    with tf.variable_scope('step6',reuse=reuse):
        net=sdconv(net,256)
    return net

def sdconv7(net):
    reuse=len([t for t in tf.global_variables() if t.name.startswith('step7')])>0
    with tf.variable_scope('step7',reuse=reuse):
        net=sdconv(net,256)
    return net

In [ ]:
x=tf.placeholder(tf.float32,[None,9,9,10])
y=tf.placeholder(tf.float32,[None,9,9,9])
net0=dconv(x,256,kernel_size=[1,1],activation=line,stride=1,norm=False)

net1=sdconv0(net0)
net=net1[:,:,:,-9:]
one=tf.square((tf.minimum(1.0,net)-1)*y)
zero=tf.square(tf.maximum(0.0,net)*(1-y))
loss10 = tf.reduce_mean(one+zero)
argmax=tf.argmax(net, -1)
correct_prediction=tf.cast(tf.equal(argmax, tf.argmax(y, -1)), tf.float32)
accuracy10 = tf.reduce_mean(correct_prediction)

net1=sdconv0(net1)
net=net1[:,:,:,-9:]
one=tf.square((tf.minimum(1.0,net)-1)*y)
zero=tf.square(tf.maximum(0.0,net)*(1-y))
loss11 = tf.reduce_mean(one+zero)
argmax=tf.argmax(net, -1)
correct_prediction=tf.cast(tf.equal(argmax, tf.argmax(y, -1)), tf.float32)
accuracy11 = tf.reduce_mean(correct_prediction)

net1=sdconv0(net1)
net=net1[:,:,:,-9:]
one=tf.square((tf.minimum(1.0,net)-1)*y)
zero=tf.square(tf.maximum(0.0,net)*(1-y))
loss12 = tf.reduce_mean(one+zero)
argmax=tf.argmax(net, -1)
correct_prediction=tf.cast(tf.equal(argmax, tf.argmax(y, -1)), tf.float32)
accuracy12 = tf.reduce_mean(correct_prediction)

net1=sdconv0(net1)
net=net1[:,:,:,-9:]
one=tf.square((tf.minimum(1.0,net)-1)*y)
zero=tf.square(tf.maximum(0.0,net)*(1-y))
loss13 = tf.reduce_mean(one+zero)
argmax=tf.argmax(net, -1)
correct_prediction=tf.cast(tf.equal(argmax, tf.argmax(y, -1)), tf.float32)
accuracy13 = tf.reduce_mean(correct_prediction)

for i in range(1,8):
    exec('net{} = sdconv{}(net{})'.format(i+1,i,i))
    for _ in range(3):
        exec('net{} = sdconv{}(net{})'.format(i+1,i,i+1))

for i in range(2,9):
    exec('net = net{}[:,:,:,-9:]'.format(i))
    one=tf.square((tf.minimum(1.0,net)-1)*y)
    zero=tf.square(tf.maximum(0.0,net)*(1-y))
    exec('loss{} = tf.reduce_mean(one+zero)'.format(i))
    argmax=tf.argmax(net, -1)
    correct_prediction=tf.cast(tf.equal(argmax, tf.argmax(y, -1)), tf.float32)
    exec('accuracy{} = tf.reduce_mean(correct_prediction)'.format(i))

'''Visualization Module'''
acchold=tf.Variable(0.0,trainable=False)
losshold=tf.Variable(0.0,trainable=False)
tf.summary.scalar('accuracy',acchold)
tf.summary.scalar('loss',losshold)

In [ ]:
rate=f.Variable(0.0001,trainable=False)
'''Pre Training'''
for i in range(4):
    exec('train1{}=tf.train.AdamOptimizer(rate).minimize(loss1{})'.format(i,i))
'''Following Training'''
for i in range(2,9):
    exec('train{}=tf.train.AdamOptimizer(rate).minimize(loss{},var_list=var)'.format(i,i))

sess=tf.Session()
merged=tf.summary.merge_all()
writer=tf.summary.FileWriter(r"board/train")
ac,cr=0,0
number=0
sess.run(tf.global_variables_initializer())
# saver=tf.train.Saver(tf.trainable_variables())
# saver.restore(sess,r'save\log')
tf.train.start_queue_runners(sess=sess)

In [ ]:
'''Pre Training'''
ra=0.0128
show=100
for step in range(4):
    for i in range(10000):
        if i%show==(show-1):
            result=sess.run(merged,feed_dict={x:feed,y:label,acchold:ac,losshold:cr})
            writer.add_summary(result,number)
            ac,cr=0,0

        feed,label=next_batch(128,num_data=40000)
        feed_dict={x:feed,y:label,rate:ra}
        exec('_,_ac,_cr=sess.run([train1{},accuracy1{},loss1{}],feed_dict=feed_dict)'.format(step,step,step))
        ac+=np.array(_ac)/show
        cr+=np.array(_cr)/show
        number+=1

In [ ]:
'''Following Training'''
for step in range(2,9):
    var1=[_ for _ in tf.trainable_variables() if 'step{}'.format(step-2) in _.name]
    var2=[_ for _ in tf.trainable_variables() if 'step{}'.format(step-1) in _.name]
    a=[tf.assign(var2[i], var1[i]) for i in range(len(var1))]
    b=sess.run(a)
    for i in range(10000):
        if i%show==(show-1):
            result=sess.run(merged,feed_dict={x:feed,y:label,acchold:ac,losshold:cr})
            writer.add_summary(result,number)
            ac,cr=0,0

        feed,label=next_batch(128,num_data=40000)
        feed_dict={x:feed,y:label,rate:ra}
        exec('_,_ac,_cr=sess.run([train{},accuracy{},loss{}],feed_dict=feed_dict)'.format(step,step,step))
        ac+=np.array(_ac)/show
        cr+=np.array(_cr)/show
        number+=1
        ra*=0.99993068768415357191320015590215
    
    

In [ ]:
saver=tf.train.Saver()
save_path=saver.save(sess,r'save\log')